# Siamese Networks for Peer Group Scoring

This notebook illustrates the application of Siamese networks to construct a peer group scoring used to create peer groups for structured products. It shows a sample implementation to reproduce results from [this presentation](siamese.pdf) (example 1).

In [1]:
import numpy as np
import tensorflow
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
%matplotlib inline

## Prepare and read data

The data is checked in as a comprezed zip folder. Therefore, we have to unzip it if we run this notebook for the first time.

In [4]:
import zipfile
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('.') #unzip into current folder

The data is organized 

### Helper Functions

In [5]:
def get_data(data, vols, data_dir):
    positive = None
    negative = None
    anchor = None
    info = None
    distance = None

    for vol in vols:
        for d in data:
            file_prefix = data_dir+d+'_' + str(vol)
            if positive is None:
                positive=pd.read_csv(file_prefix+'_positive.csv')#, index_col = 0))
                negative=pd.read_csv(file_prefix+'_negative.csv')#, index_col = 0))
                anchor=pd.read_csv(file_prefix+'_anchor.csv')#, index_col = 0))
                info=pd.read_csv(file_prefix+'_info.csv')#, index_col = 0))
                distance=pd.read_csv(file_prefix+'_distance.csv')#, index_col = 0))
            else:
                positive = positive.append(pd.read_csv(file_prefix+'_positive.csv'), ignore_index = True)#, index_col = 0))
                negative = negative.append(pd.read_csv(file_prefix+'_negative.csv'), ignore_index = True)#, index_col = 0))
                anchor = anchor.append(pd.read_csv(file_prefix+'_anchor.csv'), ignore_index = True)#, index_col = 0))
                info = info.append(pd.read_csv(file_prefix+'_info.csv'), ignore_index = True)#, index_col = 0))
                distance=distance.append(pd.read_csv(file_prefix+'_distance.csv'), ignore_index = True)#, index_col = 0))
    return anchor, positive, negative, info, distance

def get_training_data(data, vols, data_dir, shuffle = True, frac=1):
    anchor, positive, negative, info, distance = get_data(data, vols , data_dir)
    #now shuffle data
    if shuffle:
        anchor = anchor.sample(frac=frac)
        positive = positive.loc[anchor.index]
        negative = negative.loc[anchor.index]
        info = info.loc[anchor.index]
        distance = distance.loc[anchor.index]
    x_train=[anchor.values, positive.values, negative.values]
    y_train = distance.values
    return x_train, y_train, info

### ReadData

In [8]:
x_train, y_train, info = get_training_data(['EUROPEAN_P1_P1_P1','EUROPEAN_P1_P1_P2'], 
                                                   vols=[10,15,20,25,30], data_dir = './data/')

# Siamese Network with the triplet loss $\alpha$ function
Define custom inputs including the number of layers and nodes, the loss function, the data/contraints and vector of volatilities.

In [53]:
inputs = {
          'loss': siamese.SiameseNN._triplet_loss_alpha,
          'activation':'elu',
          'data':['EUROPEAN_P1_P1_P1','EUROPEAN_P1_P1_P2','BUTTERFLY_BUTTERFLY_BUTTERFLY'], 
          'vols' : [10, 15, 20, 25, 30],
          'normalize_output':False,
          'kernel_regularizer': None,
          'bias_regularizer': None,
          'optimizer':keras.optimizers.Adam(lr=0.00035, beta_1=0.9, beta_2=0.999),
          'log_dir':'C:\\temp\\siamese_alpha\\',
          'train_data_dir':'C:\\Users\\cwerner.FAV\\dev\\fav\\analytics_tools\\python\\notebooks\\fav\\peer_group\\data\\'}

Get the training data as configured within the above inputs

## Create and compile the Siamese Network Model

### Helper Functions
All functions to create the network with tensorflow/keras are encapsulated in the following code cell.

In [3]:
class SiameseNN:
    """This class encasulates all functions needed to create the siamese network for peer group scoring
    """

    @staticmethod
    def triplet_loss_alpha(y_true, y_pred):
        """
        Implementation of the triplet loss function where alpha is taken from y_true
        Arguments:
        y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
        y_pred -- python list containing three objects:
                anchor -- the encodings for the anchor data
                positive -- the encodings for the positive data (similar to anchor)
                negative -- the encodings for the negative data (different from anchor)
        Returns:
        loss -- real number, value of the loss

        $$L=\max(d(A,P)-d(A,N)+\alpha,0)$$
        """
        total_lenght = y_pred.shape[-1]

        anchor = y_pred[:,0:int(total_lenght/3)]
        positive = y_pred[:,int(total_lenght/3):int(total_lenght*2/3)]
        negative = y_pred[:,int((total_lenght*2)/3):int(total_lenght)]

        # distance between the anchor and the positive
        pos_dist = keras.backend.sum(keras.backend.square(anchor-positive),axis=1)

        # distance between the anchor and the negative
        neg_dist = keras.backend.sum(keras.backend.square(anchor-negative),axis=1)

        # compute loss
        #return keras.backend.mean(keras.backend.maximum(pos_dist-neg_dist+0.1,0.0))
        #print(keras.backend.mean(depp.shape))
        #print(keras.backend.mean(y_pred[:,3]))
        return keras.backend.maximum(pos_dist-neg_dist+0.5*y_true[:,3],0.0)

    @staticmethod
    def load(file):
        """Load a model from file.

        Thismethod simply uses the keras built-in method keras.models.load_model where the specialized loss function is specified in custom_objects

        Args:
            file (str): The filename of fil containing model

        Returns:
            tensorflow model: The siamese network
        """
        #if newest_subdir is None:
        return keras.models.load_model(file, custom_objects={'triplet_loss_alpha': SiameseNN.triplet_loss_alpha})

    @staticmethod
    def get_submodel(m):
        """Return the inner model of the siamese network

        Args:
            m (keras model): The siamese network

        Returns:
            [keras model]: The inner network of the siamese model
        """
        return m.get_layer(name='distance_model')
         
    @staticmethod
    def _create_simple_network(n_neurons, activation='relu',
                        kernel_regularizer=None, bias_regularizer=None, 
                        input_dim=1, 
                        normalize_output=False):
        """Create the network model used as inner model of the siamese network

            Args:
                n_neurons (int): Number of neurons
                activation (str, optional): The keras activation function used in the inner network. Defaults to 'relu'.
                kernel_regularizer (regularizer from keras.regularizers, optional): A possible kernel_regularizer. Defaults to None.
                bias_regularizer (regularizer from keras.regularizers, optional): A bias regularizer. Defaults to None.
                input_dim (int, optional): Input dimension. Defaults to 1.
                normalize_output (bool, optional): Flag that determines if the output is normalized. Defaults to False.

            Returns:
                [model from keras.models]: The resulting inner network
        """
        keras.backend.clear_session()
        np.random.seed(42)
        model = keras.models.Sequential(name='distance_model')
        model.add(keras.layers.Dense(n_neurons[0], activation=activation, input_dim=input_dim, kernel_regularizer=kernel_regularizer, 
                        bias_regularizer=bias_regularizer, name='distance_model_layer_0')) 
        for i,n in enumerate(n_neurons[1:]):
            name='distance_model_layer_'+str(i+1)
            model.add(keras.layers.Dense(n, activation=activation, kernel_regularizer=kernel_regularizer, 
                        bias_regularizer=bias_regularizer,  name=name)) 
        if normalize_output:
            model.add(keras.layers.Lambda(lambda t: keras.backend.l2_normalize(t, axis=1)))
        return model

    @staticmethod
    def create(n_neurons, activation='relu', kernel_regularizer=None,
            bias_regularizer=None, input_dim=1, optimizer = None, 
            normalize_output = False):
        """If optimizer is not None, it also directly compiles the model
        """
        model = SiameseNN._create_simple_network(n_neurons, activation=activation, 
                kernel_regularizer=kernel_regularizer, bias_regularizer=bias_regularizer, 
                input_dim=input_dim, normalize_output=normalize_output)
        input_dim = model.layers[0].input_shape[1]
        # Define the tensors for the three input images
        anchor_input = keras.layers.Input((input_dim, ), name="anchor_input")
        positive_input = keras.layers.Input((input_dim, ), name="positive_input")
        negative_input = keras.layers.Input((input_dim, ), name="negative_input") 
        
        # Generate the encodings (feature vectors) for the three images
        encoded_a = model(anchor_input)
        encoded_p = model(positive_input)
        encoded_n = model(negative_input)
        
        merged_vector = keras.layers.concatenate([encoded_a, encoded_p, encoded_n], axis=-1, name='merged_layer')
        #merged_vector = keras.backend.stack([encoded_a, encoded_p, encoded_n])
        # Connect the inputs with the outputs
        siamese = keras.models.Model(inputs=[anchor_input,positive_input,negative_input],outputs=merged_vector, name='siamese')
        if optimizer is not None:
            siamese.compile(loss=SiameseNN._triplet_loss, optimizer=optimizer)
        return siamese

### Creating the Siamese Network

In the next code cell we define a new Siamese network with
- Three layers (15 neurons in the first layer, 10 in the second and 3 in the output layer)
- As activation functions we use elu
- We do not apply any regularization

In [55]:
model = siamese.SiameseNN.create(n_neurons=(15,10,3,), 
                                 activation='elu',
                                 kernel_regularizer=None, 
                                 bias_regularizer=None, 
                                 input_dim =x_train[0].shape[1], 
                                 normalize_output = False)

model.compile(loss=siamese.SiameseNN.triplet_loss_alpha, 
              optimizer = keras.optimizers.Adam(lr=0.00035, beta_1=0.9, beta_2=0.999))

### Fit the model:

In [56]:
cb = [keras.callbacks.TensorBoard(profile_batch=0, log_dir=inputs['log_dir'], histogram_freq=100),]
cb.append(keras.callbacks.ModelCheckpoint(inputs['log_dir']+'\\' + dt.datetime.now().strftime("%Y%m%d-%H%M%S") + '\\best_model.h5', save_best_only = True))


model.summary()
history = model.fit(x_train,
                    y_train,
                    epochs=50,
                    batch_size=2000,
                    verbose=0,
                    callbacks=cb,
                    validation_split=0.2)
#save model
model.reset_metrics()
model.save(inputs['log_dir'] +'\\model.h5')

Model: "siamese"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       [(None, 33)]         0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     [(None, 33)]         0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     [(None, 33)]         0                                            
__________________________________________________________________________________________________
distance_model (Sequential)     (None, 3)            703         anchor_input[0][0]               
                                                                 positive_input[0][0]       

AttributeError: 'OSError' object has no attribute 'message'

In [50]:
import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
history[0]

TypeError: 'History' object is not subscriptable